# Collect game information

We will be using information from http://survivor.wikia.com

## Dependencies

In [1]:
from bs4 import BeautifulSoup
import urllib2
import re

import numpy as np
import pandas as pd
import itertools as iter
from IPython import display as dis

In [2]:
# silly utility to launch a qtconsole if one doesn't exist
import psutil

def returnPyIDs():
    pyids = set()
    for pid in psutil.pids():
        try:
            if "python" in psutil.Process(pid).name():
                pyids.add(pid)
        except:
            pass
    return pyids

def launchConsole():
    before_pyids = returnPyIDs()
    %qtconsole
    after_pyids = returnPyIDs()
    newid = after_pyids.difference(before_pyids)
    assert len(newid) == 1
    return list(newid)[0]

try:
    qtid
except NameError:
    qtid = launchConsole()
    
if qtid not in returnPyIDs():
    qtid = launchConsole()
    
qtid

5384

In [96]:
def parse_td(td):
    # Extract text from table data element
    try:
        # if there is a colspan element present ...
        n = int(td['colspan'])
    except KeyError:
        n = 1
    # (these all end with \n so strip that)
    return [td.text[:-1]] * n  # ... repeat text n times

def get_voting_results(url):
    html= urllib2.urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')
    voting_table = soup.find(id='Voting_History').find_next('table')
    
    # Eliminated
    eliminated = [i.text[:-1] for i in voting_table
                  .find('th', text=re.compile(r"(Voted Out|Eliminated)"))
                  .findPrevious('tr').findAll('td')]
    
    # we want all rows after the row that starts with "Vote:"
    voting_rows = voting_table.find('th', text=re.compile(r"Vote:")).findAllNext('tr')
    # extract the text from all table data in the rows we just extracted
    votes = [list(chain(*[parse_td(i) for i in j.findAll('td')])) for j in voting_rows]

    # convert to pandas data frame
    votes = pd.DataFrame(votes)
    # the contestant casting the vote is in the second row - make that the index
    votes.index = votes[1]
    # now we can drop the first two columns
    votes.drop(votes.columns[[0, 1]], inplace=True, axis=1)
    # drop any rows where the index is None
    votes.drop(votes.index[[i == None for i in votes.index]], inplace=True, axis=0)
    # Our columns correspond to those eliminated (from above)
    # There appears to be an extra column. Drop anthing beyoned the elimination info we have
    votes = votes[votes.columns[[range(len(eliminated))]]]
    # rename columns to be the eliminated contestants
    votes.columns = eliminated
    
    # Create a contestant by contestant matrix corresponding to the number of like votes
    like_votes = np.array([None] * votes.shape[0]**2).reshape((votes.shape[0], votes.shape[0]))

    for (i, j) in iter.combinations(range(votes.shape[0]), 2):
        like_votes[i, j] = sum(votes.iloc[i, :] == votes.iloc[j, :])

    like_votes = pd.DataFrame(like_votes)
    like_votes.index = votes.index
    like_votes.columns = votes.index
    return like_votes

In [3]:
url = "http://survivor.wikia.com/wiki/Survivor:_Cambodia"
get_voting_results(url)

1,Abi-Maria,Jeremy,Keith,Kelley,Kimmi,Spencer,Tasha,Joe,Stephen,Ciera,Kelly,Andrew,Kass,Woo,Terry,Monica,Jeff,Peih-Gee,Shirin,Vytas
1,,,,,,,,,,,,,,,,,,,,
Abi-Maria,None,4,5,9,3,8,7,3,3,5,1,4,4,3,2,0,2,1,2,0
Jeremy,None,None,8,7,11,8,7,8,11,6,5,4,5,1,2,4,0,0,0,0
Keith,None,None,None,8,9,7,9,10,8,6,4,6,6,2,3,4,0,0,0,0
Kelley,None,None,None,None,7,8,6,5,7,7,3,3,4,2,4,2,1,1,2,0
Kimmi,None,None,None,None,None,6,10,7,12,5,5,5,5,1,2,4,0,0,0,0
Spencer,None,None,None,None,None,None,5,6,6,5,6,2,4,2,2,3,2,2,1,0
Tasha,None,None,None,None,None,None,None,7,8,4,3,8,4,3,1,2,1,0,0,0
Joe,None,None,None,None,None,None,None,None,8,6,5,5,6,2,3,4,0,0,0,0
Stephen,None,None,None,None,None,None,None,None,None,6,4,5,5,1,2,4,0,0,0,0


TODO:

* some way to normalize.
* wrap this in a function that accepts a web page as an argument and returns a DataFrame

In [4]:
url = "http://survivor.wikia.com/wiki/Main_Page"
html= urllib2.urlopen(url)
soup = BeautifulSoup(html, 'html.parser')

In [5]:
prefix = "http://survivor.wikia.com"
seasons = set([i['href'] for i in soup.findAll(
            'a', {'href': re.compile("^/wiki/Survivor:")}
        )])
seasons = [prefix + i for i in list(seasons)]
seasons

[u'http://survivor.wikia.com/wiki/Survivor:_Nicaragua',
 u'http://survivor.wikia.com/wiki/Survivor:_Gabon',
 u'http://survivor.wikia.com/wiki/Survivor:_Tocantins',
 u'http://survivor.wikia.com/wiki/Survivor:_One_World',
 u'http://survivor.wikia.com/wiki/Survivor:_China',
 u'http://survivor.wikia.com/wiki/Survivor:_Redemption_Island',
 u'http://survivor.wikia.com/wiki/Survivor:_Thailand',
 u'http://survivor.wikia.com/wiki/Survivor:_The_Amazon',
 u'http://survivor.wikia.com/wiki/Survivor:_Pearl_Islands',
 u'http://survivor.wikia.com/wiki/Survivor:_Heroes_vs_Villains',
 u'http://survivor.wikia.com/wiki/Survivor:_San_Juan_del_Sur',
 u'http://survivor.wikia.com/wiki/Survivor:_The_Australian_Outback',
 u'http://survivor.wikia.com/wiki/Survivor:_Cambodia',
 u'http://survivor.wikia.com/wiki/Survivor:_Palau',
 u'http://survivor.wikia.com/wiki/Survivor:_Cagayan',
 u'http://survivor.wikia.com/wiki/Survivor:_Samoa',
 u'http://survivor.wikia.com/wiki/Survivor:_Caramoan',
 u'http://survivor.wikia.co

In [97]:
for url in seasons:
    try:
        print url
        dis.display(get_voting_results(url))
    except AttributeError, e:
        print e
    except IndexError, e:
        print e

http://survivor.wikia.com/wiki/Survivor:_Nicaragua


1,Fabio,Chase,Sash,Holly,Dan,Jane,Benry,Kelly S.,NaOnka,Brenda,Marty,Alina,Jill,Yve,Kelly B.,Tyrone,Jimmy T.,Jimmy J.,Shannon,Wendy
1,,,,,,,,,,,,,,,,,,,,
Fabio,None,10,13,6,6,6,8,8,5,8,4,4,3,0,4,0,0,0,2,0
Chase,None,None,13,13,10,5,13,6,12,6,1,8,0,2,4,0,0,0,1,0
Sash,None,None,None,8,5,8,8,10,7,10,3,3,3,0,6,0,0,0,1,0
Holly,None,None,None,None,14,11,10,3,9,4,5,7,4,6,1,4,5,3,1,1
Dan,None,None,None,None,None,8,14,4,10,3,9,7,7,9,3,7,6,5,3,3
Jane,None,None,None,None,None,None,6,6,5,7,8,5,8,6,5,6,7,5,3,3
Benry,None,None,None,None,None,None,None,10,16,9,7,12,5,7,9,5,5,5,6,5
Kelly S.,None,None,None,None,None,None,None,None,14,16,9,8,9,6,11,6,6,6,7,6
NaOnka,None,None,None,None,None,None,None,None,None,11,7,13,6,8,10,6,6,6,7,6


http://survivor.wikia.com/wiki/Survivor:_Gabon


1,Bob,Susie,Sugar,Matty,Ken,Crystal,Corinne,Randy,Charlie,Marcus,Dan,Ace,Kelly,G.C.,Jacquie,Paloma,Gillian,Michelle
1,,,,,,,,,,,,,,,,,,
Bob,None,7,6,2,1,0,11,6,9,8,4,3,2,0,3,2,0,0
Susie,None,None,5,9,6,7,5,7,5,4,7,0,0,3,0,0,1,0
Sugar,None,None,None,9,6,7,4,1,4,3,0,5,3,0,3,2,0,0
Matty,None,None,None,None,12,13,2,5,2,1,5,5,4,6,2,2,3,2
Ken,None,None,None,None,None,16,3,5,2,2,6,5,5,7,3,3,4,3
Crystal,None,None,None,None,None,None,3,6,3,3,7,6,6,8,4,4,5,4
Corinne,None,None,None,None,None,None,None,13,16,13,9,8,7,5,8,7,5,5
Randy,None,None,None,None,None,None,None,None,13,12,14,6,6,9,6,6,7,6
Charlie,None,None,None,None,None,None,None,None,None,15,11,10,9,7,10,9,7,7


http://survivor.wikia.com/wiki/Survivor:_Tocantins


...

1,Sandra,Parvati,Russell,Jerri,Colby,Rupert,Danielle,Candice,Amanda,J.T.,Courtney,Coach,Rob,James,Tyson,Tom,Cirie,Randy,Stephenie,Sugar
1,,,,,,,,,,,,,,,,,,,,
Sandra,None,14,13,12,1,1,11,1,0,0,8,6,8,0,5,0,0,2,0,0
Parvati,None,None,14,12,0,0,14,2,0,0,7,6,6,0,5,0,0,2,0,0
Russell,None,None,None,15,2,2,11,1,0,0,6,7,6,0,5,0,0,2,0,0
Jerri,None,None,None,None,3,3,13,2,1,1,8,11,8,2,8,2,2,4,2,2
Colby,None,None,None,None,None,16,2,12,13,13,4,2,3,6,3,8,5,3,5,3
Rupert,None,None,None,None,None,None,3,15,16,15,5,3,4,10,4,6,7,4,5,4
Danielle,None,None,None,None,None,None,None,6,4,4,11,12,11,5,10,5,5,7,5,5
Candice,None,None,None,None,None,None,None,None,18,17,7,5,6,11,6,8,10,6,7,6
Amanda,None,None,None,None,None,None,None,None,None,18,8,6,7,12,7,9,11,7,8,7


http://survivor.wikia.com/wiki/Survivor:_Worlds_Apart
'NoneType' object has no attribute 'findPrevious'
http://survivor.wikia.com/wiki/Survivor:_Panama


1,Aras,Danielle,Terry,Cirie,Shane,Courtney,Bruce,Sally,Austin,Nick,Dan,Bobby,Ruth-Marie,Misty,Melinda,Tina
1,,,,,,,,,,,,,,,,
Aras,None,11,3,10,9,9,7,1,1,1,1,5,0,1,0,0
Danielle,None,None,3,10,9,10,7,1,1,1,1,4,0,1,0,0
Terry,None,None,None,3,2,5,1,9,10,8,5,2,3,3,1,1
Cirie,None,None,None,None,8,11,7,3,3,2,2,4,3,2,4,2
Shane,None,None,None,None,None,11,11,3,3,4,4,7,3,4,3,3
Courtney,None,None,None,None,None,None,10,6,6,5,5,8,4,5,4,4
Bruce,None,None,None,None,None,None,None,5,5,6,6,8,5,6,5,5
Sally,None,None,None,None,None,None,None,None,13,11,9,7,7,9,6,6
Austin,None,None,None,None,None,None,None,None,None,14,11,8,9,9,7,7


http://survivor.wikia.com/wiki/Survivor:_Vanuatu


1,Chris,Twila,Scout,Eliza,Julie,Ami,Leann,Chad,Lea,Rory,John K.,Lisa,Travis,Brady,Mia,John P.,Dolly,Brook
1,,,,,,,,,,,,,,,,,,
Chris,None,8,5,3,4,1,1,10,9,6,5,0,5,2,0,1,0,0
Twila,None,None,13,9,9,8,8,3,3,0,2,4,0,0,2,0,1,0
Scout,None,None,None,11,6,12,10,0,0,3,1,7,1,1,3,1,2,1
Eliza,None,None,None,None,11,11,13,3,3,5,2,6,2,2,5,2,3,2
Julie,None,None,None,None,None,10,11,7,7,3,5,7,3,3,7,3,5,3
Ami,None,None,None,None,None,None,15,3,3,7,4,9,4,4,6,4,5,4
Leann,None,None,None,None,None,None,None,6,6,8,5,10,5,5,7,5,6,5
Chad,None,None,None,None,None,None,None,None,16,12,11,6,11,8,6,7,6,6
Lea,None,None,None,None,None,None,None,None,None,13,12,7,12,9,7,8,7,7


In [95]:
def parse_td(td):
    try:
        # if there is a colspan element present ...
        n = int(td['colspan'])
    except KeyError:
        n = 1
    # (these all end with \n so strip that)
    return [td.text[:-1]] * n  # ... repeat text n times

url = 'http://survivor.wikia.com/wiki/Survivor:_Nicaragua'
html= urllib2.urlopen(url)
soup = BeautifulSoup(html, 'html.parser')
voting_table = soup.find(id='Voting_History').find_next('table')

# Eliminated
eliminated = [i.text[:-1] for i in voting_table
              .find('th', text=re.compile(r"(Voted Out|Eliminated)"))
              .findPrevious('tr').findAll('td')]

# we want all rows after the row that starts with "Vote:"
voting_rows = voting_table.find('th', text=re.compile(r"Vote:")).findAllNext('tr')
# extract the text from all table data in the rows we just extracted
votes = [list(chain(*[parse_td(i) for i in j.findAll('td')])) for j in voting_rows]

# convert to pandas data frame
votes = pd.DataFrame(votes)
# the contestant casting the vote is in the second row - make that the index
votes.index = votes[1]
# now we can drop the first two columns
votes.drop(votes.columns[[0, 1]], inplace=True, axis=1)
# drop any rows where the index is None
votes.drop(votes.index[[i == None for i in votes.index]], inplace=True, axis=0)
# Our columns correspond to those eliminated (from above)
# There appears to be an extra column. Drop anthing beyoned the elimination info we have
votes = votes[votes.columns[[range(len(eliminated))]]]
# rename columns to be the eliminated contestants
votes.columns = eliminated

# Create a contestant by contestant matrix corresponding to the number of like votes
like_votes = np.array([None] * votes.shape[0]**2).reshape((votes.shape[0], votes.shape[0]))

for (i, j) in iter.combinations(range(votes.shape[0]), 2):
    like_votes[i, j] = sum(votes.iloc[i, :] == votes.iloc[j, :])

like_votes = pd.DataFrame(like_votes)
like_votes.index = votes.index
like_votes.columns = votes.index
dis.display(votes)
# print voting_table.prettify()

,Wendy,Shannon,Jimmy J.,Jimmy T.,Tyrone,Kelly B.,Yve,Jill,Alina,Marty,Brenda,NaOnka,Kelly S.,Benry,Jane,Dan,Holly,Sash,Chase,Fabio
1,,,,,,,,,,,,,,,,,,,,
Fabio,—,Brenda,—,—,—,Kelly B.,Kelly B.,—,Marty,Alina,Jane,Brenda,—,—,Holly,Jane,Dan,Holly,None,None
Chase,—,Shannon,—,—,Tyrone,—,—,Yve,—,Alina,Marty,Brenda,—,—,Benry,Jane,Dan,Holly,None,None
Sash,—,Shannon,—,—,—,Marty,Kelly B.,—,Jill,Alina,Marty,Brenda,—,—,Benry,Jane,Dan,Holly,None,None
Holly,Wendy,—,Jimmy J.,Dan,Tyrone,—,—,Yve,—,Alina,Marty,Brenda,—,—,Benry,Jane,Dan,Sash,,Chase
Dan,Wendy,—,Jimmy J.,Jimmy T.,Tyrone,—,—,Yve,—,Alina,Jane,Brenda,—,—,Fabio,Jane,Chase,,,
Jane,Wendy,—,Jimmy J.,Dan,—,Marty,Marty,—,Marty,Marty,Marty,Brenda,—,—,Benry,Sash,,,,Chase
Benry,—,Shannon,—,—,Tyrone,—,—,Yve,—,Alina,Jane,Brenda,—,—,Fabio,,,,,
Kelly S.,—,Shannon,—,—,—,Kelly B.,Kelly B.,—,Jill,Alina,Marty,Benry,Quit,Quit,,,,,,
NaOnka,—,Shannon,—,—,Tyrone,—,—,Yve,—,Alina,Marty,Brenda,Quit,Quit,,,,,,
